In [1]:
import yaml
from omegaconf import OmegaConf
from yolo_ev.module.module import Module

yaml_file = "./config/param.yaml"
with open(yaml_file, 'r') as file:
    config = yaml.safe_load(file)
config = OmegaConf.create(config)
module = Module(config)